# Fall 2022 Data Science Intern Challenge 


# Question 1: 

Given some sample data, write a program to answer the following: click here to access the required data set

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
What metric would you report for this dataset?
What is its value?


In [18]:
# Importing required files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Mounting the google drive to import the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Reading the data into pandas dataframe
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Shopify Intern Challenge/2019 Winter Data Science Intern Challenge Data Set.csv')
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [12]:
# Doing a quick sanity check
print("AOV as per the method provided in the question is {}".format(round(sum(df.order_amount)/len(df.order_amount))))

AOV as per the method provided in the question is 3145


# Solution 1a :
The problem with this calculation is the following:

1.   The average is not taking into account the quantity of shoes that are being sold on a particular day.

2.   Average as a statistic is inherently prone to outliers. Extremely high or low values could skew the average towards their direction



In [14]:
# The orders having high amount

df[df['order_amount']>1000]

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57
490,491,78,936,51450,2,debit,2017-03-26 17:08:19
493,494,78,983,51450,2,cash,2017-03-16 21:39:35
...,...,...,...,...,...,...,...
4646,4647,42,607,704000,2000,credit_card,2017-03-02 4:00:00
4715,4716,78,818,77175,3,debit,2017-03-05 5:10:44
4868,4869,42,607,704000,2000,credit_card,2017-03-22 4:00:00
4882,4883,42,607,704000,2000,credit_card,2017-03-25 4:00:00


# Observations:

On inspecting the above rows 

1. Store 42 is selling a large amount of sneakers approximately 2000 in some of the orders. Probably a bulk order has been placed by some other local retailer. 
2. Store 78 is selling a pair of sneaker at a really high price - 25,725 compared to all other stores

These orders could sway the average and result in a number which is not realistic for any kind of business decisions





# Solution 1b

In order to evaluate the data correctly such that the metric reports the right AOV that could be useful for making useful decisions we need to use a statistic that is not prone to outliers like Median. However, in this case we need to consider the quantity sold as well. Thus we can do the following:

1.   Find price of one pair of sneakers by dividing the order_amount by total_items sold in each individual orders

2.   Find the median across this newly created metric. The average would still sway since there is a store that is selling one pair of shoes at 25k. 





In [15]:
# Creating column with price of one pair in every order
df['Price_per_shoe'] = df['order_amount'] / df['total_items']
df['Price_per_shoe']

0       112.0
1        90.0
2       144.0
3       156.0
4       156.0
        ...  
4995    165.0
4996    117.0
4997    117.0
4998    177.0
4999    144.0
Name: Price_per_shoe, Length: 5000, dtype: float64

In [19]:
# Median price across all the shoes

np.median(df['Price_per_shoe'])

153.0

# Solution 1c

The median value of price per shoe across all the orders comes out to be $153$. This value could help in understanding which stores are selling the shoes at very low or higher prices. 


#Question 2: 

For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.



1.   How many orders were shipped by Speedy Express in total?
2.   What is the last name of the employee with the most orders?
3.   What product was ordered the most by customers in Germany?







# Solution 2a:

SQL Query:

select count(*) as Number_of_Orders

from Orders A 

left join Shippers B

on A.ShipperID = B.ShipperID

where ShipperName = 'Speedy Express';

Output:

The total number of orders shipped by Speedy Express is $54$





# Solution 2b:

SQL Query:

select B.LastName 

from 

(select EmployeeID,count(OrderID) as Number_of_Orders 

from Orders

group by EmployeeID

order by Number_of_Orders desc

limit 1) A

left join 

Employees B

on A.EmployeeID = B.EmployeeID;


Output:


The last name of the employee who has the maximum number of orders (40) is Peacock





# Solution 2c:

SQL Query:

SELECT ProductName

FROM Orders

left join Customers

on Orders.CustomerID = Customers.CustomerID

left join OrderDetails 

on Orders.OrderID = OrderDetails.OrderID

left join Products 

on OrderDetails.ProductID = Products.ProductID

where Country = 'Germany'

group by ProductName

order by sum(Quantity) desc

limit 1;



Output:


The product that was ordered the most by customers in Germany was Boston Crab Meat (this is based on the quantity ordered by the customers) 